In [18]:
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_community.vectorstores import FAISS# Chroma
from langchain_openai import OpenAIEmbeddings
from dotenv import load_dotenv
import warnings

warnings.filterwarnings("ignore")
load_dotenv()

True

In [3]:
# from langchain_community.document_loaders import TextLoader

# from langchain.text_splitter import CharacterTextSplitter
# from langchain_community.vectorstores import Chroma


# #텍스트를 600자 단위로 분할
# text_splitter = CharacterTextSplitter(chunk_size=600, chunk_overlap=0)

# # TextLoader 를 통해 텍스트 파일을 로드
# split_docs = TextLoader("data/ko_script.txt").load_and_split(text_splitter)

# # Chroma 를 통해 벡터 저장소 생성
# chroma_db = Chroma.from_documents(split_docs, OpenAIEmbeddings())


In [29]:
# OpenAI의 "text-embedding-3-large" 모델을 사용하여 임베딩을 생성합니다.
embeddings = OpenAIEmbeddings(model="text-embedding-3-large")

In [23]:
loader = PyMuPDFLoader("data/ko_script.pdf")
docs = loader.load()
print(f"문서의 페이지수: {len(docs)}")

문서의 페이지수: 50


In [26]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
# 단계 2: 문서 분할(Split Documents)
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=50)
split_documents = text_splitter.split_documents(docs)
print(f"분할된 청크의수: {len(split_documents)}")


분할된 청크의수: 50


In [28]:
split_documents[2]

Document(metadata={'source': 'data/ko_script.pdf', 'file_path': 'data/ko_script.pdf', 'page': 2, 'total_pages': 50, 'format': 'PDF 1.7', 'title': '', 'author': '신혁준[ 학부수료 / 국어국문학과 ]', 'subject': '', 'keywords': '', 'creator': 'Microsoft® Word Microsoft 365용', 'producer': 'Microsoft® Word Microsoft 365용', 'creationDate': "D:20240713090111+09'00'", 'modDate': "D:20240713090111+09'00'", 'trapped': ''}, page_content='(마지의 몸 커진다) \n마지 : 버논! 버논! 버논! 어떻게 좀 해 봐요. \n버논 : 잡았어, 마지. 잡았다고. \n마지 : 놓지 말아요. \n버논 : 미안. 마지! 제발! 마지! 마지! \n(해리 그의 가방을 들고 방에서 나온다) \n버논 : 그녀를 정상으로 돌려놔! 정상으로, 지금 돌려놔! \n해리 : 안돼요! 아줌마가 자초했어요. \n(버논 소리친다) \n(해리 지팡이를 버논을 향해 든다) \n해리 : 물러서요! \n버논 : 너도 학교 밖에서 마법을 쓰지 못한다는 것을 알잖아? \n해리 : 그래요? 해 볼까요? \n버논 : 그들은 다시는 학교에 널 받아주지 않을거야. 넌 갈데도 없어. \n해리 : 상관없어요. 어디든 여기보단 나요. \n(해리 가방을 들고 길을 걷는다. 마지 창공을 떠다니고 있다. 무서운 분위기. 검은 개 해리에게 짖\n는다. 해 \n리 지팡이를 든다. 해리 넘어진다. 구조 버스 온다) \n스탠 : 구조 버스에 오신 것을 환영합니다, 급하거나 갈 때 없는 마녀나 마법사들을 위해 이동합\n니다. 제  \n이름은 스탠 션파이크고, 저는 오늘 밤 여러분을 모실 차장입니다. 거기서 뭐하고 있어? \n해리 : 넘

In [35]:
# 단계 3: 임베딩(Embedding) 생성
embeddings = OpenAIEmbeddings()

In [31]:
# 단계 4: DB 생성(Create DB) 및 저장
# 벡터스토어를 생성합니다.
vectorstore = FAISS.from_documents(documents= split_documents[1:2], embedding=embeddings)


In [32]:
rt = vectorstore.as_retriever()

In [33]:
rt.invoke('버논 : 성 부르스터스. 가장 좋은 선택이었지.')

[Document(metadata={'source': 'data/ko_script.pdf', 'file_path': 'data/ko_script.pdf', 'page': 1, 'total_pages': 50, 'format': 'PDF 1.7', 'title': '', 'author': '신혁준[ 학부수료 / 국어국문학과 ]', 'subject': '', 'keywords': '', 'creator': 'Microsoft® Word Microsoft 365용', 'producer': 'Microsoft® Word Microsoft 365용', 'creationDate': "D:20240713090111+09'00'", 'modDate': "D:20240713090111+09'00'", 'trapped': ''}, page_content='버논 : 한 잔 할래, 마지? \n마지 : 조금만요, 맛있는 식사였어요, 언니. 그래요, 조금만 더요. 12마리의 개들과 있으면 식사는 \n거르기 \n가 일수죠. 조금만 더요, 됐어요. 브랜디 좀 마셔 볼래? 브랜디 조금만... 뭘 히죽거리니? 저 아이\n를 어디에  \n보내고 있죠, 오빠? \n버논 : 성 부르스터스. 가장 좋은 선택이었지. \n마지 : 성 부르스터스에서는 체벌도 하니 얘야? \n해리 : 오, 그럼요. 전 많이 맞아요. \n마지 : 좋아. 난 저런 부류의 애들이 다니는 학교는 가지 않을 거예요. 저 아이가 저렇게 된 것은 \n오빠 탓 \n이 아니에요, 오빠. 그건 유전이죠. 나쁜 피가 흐른거죠. 아이의 아버지는 뭘 했죠, 언니? \n페투니아 : 아무것도. 그는 아무것도 안했어요. 실직 상태였죠. \n마지 : 주정뱅이 였죠? \n해리 : 거짓말 이예요! \n마지 : 뭐라고 했지? \n해리 : 내 아버지는 주정뱅이가 아니였어요! \n(유리 깨진다) \n마지 : 걱정 할 필요 없어요, 내가 너무 세게 잡았나봐요, 언니. 내가 세게 잡았나... \n버논 : 네가 잠자리에 들 시간이 된 것 같구나. \n마지 : 가만히

In [25]:
vectorstore.save_local("db/kor_openai_ver1")

In [20]:
# 로컬에 저장된 데이터베이스를 불러와 new_db 변수에 할당합니다.
local_db = FAISS.load_local('db/kor_openai_ver1', embeddings,
                          allow_dangerous_deserialization=True)

query = "버논 : 그녀를 정상으로 돌려놔! 정상으로, 지금 돌려놔!"

# # new_db에서 query와 유사한 문서를 검색하여 docs 변수에 할당합니다.
# docs = new_db.similarity_search(query)

# # 문서 리스트의 첫 번째 문서를 가져옵니다.
# docs[0]


In [21]:
local_retriever = local_db.as_retriever()

In [22]:
local_retriever.invoke('론 : 닥쳐 말포이')[0].page_content

AssertionError: 